## 1. making a RNN pretrained model

In [2]:
import os
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.layers import Bidirectional
from keras.preprocessing import sequence
from keras.layers import Dropout
from keras.models import model_from_json
from keras.models import load_model


In [3]:
import os
import random
import numpy as np
import pandas as pd

from tensorflow.python.keras.preprocessing import sequence
from tensorflow.python.keras.preprocessing import text

def load_imdb_sentiment_analysis_dataset(data_path, seed=123):
    imdb_data_path = os.path.join(data_path, 'aclImdb')
    print(imdb_data_path)

    # Load the training data
    train_texts = []
    train_labels = []
    for category in ['pos', 'neg']:
        train_path = os.path.join(imdb_data_path, 'train', category)
        for fname in sorted(os.listdir(train_path)):
            if fname.endswith('.txt'):
                with open(os.path.join(train_path, fname)) as f:
                    train_texts.append(f.read())
                train_labels.append(0 if category == 'neg' else 1)

    # Load the validation data.
    #test_texts = []
    #test_labels = []
    for category in ['pos', 'neg']:
        test_path = os.path.join(imdb_data_path, 'test', category)
        for fname in sorted(os.listdir(test_path)):
            if fname.endswith('.txt'):
                with open(os.path.join(test_path, fname)) as f:
                    train_texts.append(f.read())
                train_labels.append(0 if category == 'neg' else 1)

    # Shuffle the training data and labels.
    random.seed(seed)
    random.shuffle(train_texts)
    random.seed(seed)
    random.shuffle(train_labels)

    return (train_texts, np.array(train_labels) )



def sequence_vectorize(train_texts):
    # Vectorization parameters
    # Limit on the number of features. We use the top 20K features.
    TOP_K = 20000
    # Limit on the length of text sequences. Sequences longer than this
    # will be truncated.
    MAX_SEQUENCE_LENGTH = 500
    # Create vocabulary with training texts.
    tokenizer = text.Tokenizer(num_words=TOP_K)
    tokenizer.fit_on_texts(train_texts)

    # Vectorize training and validation texts.
    x_train = tokenizer.texts_to_sequences(train_texts)
    #x_val = tokenizer.texts_to_sequences(val_texts)

    # Get max sequence length.
    max_length = len(max(x_train, key=len))
    if max_length > MAX_SEQUENCE_LENGTH:
        max_length = MAX_SEQUENCE_LENGTH

    # Fix sequence length to max value. Sequences shorter than the length are
    # padded in the beginning and sequences longer are truncated
    # at the beginning.
    x_train = sequence.pad_sequences(x_train, maxlen=max_length)
    #x_val = sequence.pad_sequences(x_val, maxlen=max_length)
    return x_train, tokenizer


    

In [4]:
#load the data
# get the sequences
#define useful global variables
import pickle
TOP_K = 20000
MAX_SEQUENCE_LENGTH  = 500
EMBEDDING_DIM = 100

data_path = '/home/rohit/Documents/Study/Projects/HACKATHON INNOVATE FOR IIT/aclImdb_v1'
(train_texts, train_labels) = load_imdb_sentiment_analysis_dataset(data_path)

print('train_text length ', len(train_texts))
print(train_labels.shape)
#sequences

    
x_train, tokenizer = sequence_vectorize(train_texts)

with open('tokenizer_on_IMDB.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


num_words = min(TOP_K, len(tokenizer.word_index) + 1)

/home/rohit/Documents/Study/Projects/HACKATHON INNOVATE FOR IIT/aclImdb_v1/aclImdb
train_text length  50000
(50000,)


In [5]:
#get the glove embeedings
glove_dir = '/home/rohit/Documents/Study/Projects/HACKATHON INNOVATE FOR IIT/'
embeddings_index = {}
with open(os.path.join(glove_dir, 'glove.6B.100d.txt')) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in tokenizer.word_index.items():
    if i >= TOP_K:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector


Found 400000 word vectors.


In [6]:
weight_matrix = embedding_matrix
max_words = MAX_SEQUENCE_LENGTH

In [7]:
def create_model_rnn(weight_matrix, max_words, EMBEDDING_DIM):
    # create the model
    model = Sequential()
    model.add(Embedding(len(weight_matrix), EMBEDDING_DIM, weights=[weight_matrix], input_length=max_words, trainable=False))
    model.add(Bidirectional(LSTM(128, dropout=0.4, recurrent_dropout=0.2)))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.50))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [8]:
# Adam Optimiser
#some hyperparameters
model_rnn = create_model_rnn(weight_matrix = embedding_matrix, max_words= MAX_SEQUENCE_LENGTH, EMBEDDING_DIM = 100)

learning_rate=1e-3,
epochs=1000,
batch_size= 128,
blocks=2,
filters=64,
dropout_rate= 0.5,
embedding_dim=EMBEDDING_DIM,
kernel_size=3,
pool_size=3

num_classes = 2
num_features = num_words
loss = 'binary_crossentropy'
optimizer = keras.optimizers.Adam(lr=learning_rate)
model_rnn.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [9]:
model_rnn.fit(x_train,train_labels, epochs= 10, verbose = 1, batch_size = 256)

Epoch 1/10
50000/50000 [==============================] - 168s 3ms/step - loss: 0.6347 - acc: 0.6321
Epoch 2/10
50000/50000 [==============================] - 167s 3ms/step - loss: 0.5062 - acc: 0.7529
Epoch 3/10
50000/50000 [==============================] - 166s 3ms/step - loss: 0.4018 - acc: 0.8183
Epoch 4/10
50000/50000 [==============================] - 166s 3ms/step - loss: 0.3652 - acc: 0.8387
Epoch 5/10
50000/50000 [==============================] - 166s 3ms/step - loss: 0.3455 - acc: 0.8495
Epoch 6/10
50000/50000 [==============================] - 168s 3ms/step - loss: 0.3288 - acc: 0.8574
Epoch 7/10
50000/50000 [==============================] - 168s 3ms/step - loss: 0.3182 - acc: 0.8624
Epoch 8/10
50000/50000 [==============================] - 172s 3ms/step - loss: 0.3083 - acc: 0.8676
Epoch 9/10
50000/50000 [==============================] - 177s 4ms/step - loss: 0.3030 - acc: 0.8723
Epoch 10/10
50000/50000 [==============================] - 174s 3ms/step - loss: 0.2944 - a

In [10]:
model_rnn.save("model_rnn_pretrained.h5")


## training the pretrained model

In [11]:
#data

df = pd.read_csv('./Reviews Data/train.csv', index_col= 0)
#df_test = pd.read_csv('./Reviews Data/data_test1.csv', index_col = 0)
#df.head(10)

df_train_reviews = list(df.text[:9000])
df_train_labels = np.asarray(df.funny[:9000]).reshape(len(df_train_reviews), 1)

df_test_reviews = list(df.text[9000:])
df_test_labels = np.asarray(df.funny[9000:]).reshape(len(df_test_reviews),1)

In [12]:
from keras.models import load_model
model_pretrained = load_model('model_rnn_pretrained.h5')



In [15]:
from tensorflow.python.keras.preprocessing import sequence
from tensorflow.python.keras.preprocessing import text

def sequence_vectorize(train_texts, val_texts):
    # Create vocabulary with training texts.
    MAX_SEQUENCE_LENGTH = 500
    
    tokenizer = text.Tokenizer(num_words=TOP_K)
    tokenizer.fit_on_texts(train_texts)
    
    x_train = tokenizer.texts_to_sequences(train_texts)
    x_val = tokenizer.texts_to_sequences(val_texts)

    # Get max sequence length.
    max_length = len(max(x_train, key=len))
    if max_length > MAX_SEQUENCE_LENGTH:
        max_length = MAX_SEQUENCE_LENGTH

    # Fix sequence length to max value. Sequences shorter than the length are
    # padded in the beginning and sequences longer are truncated
    # at the beginning.
    x_train = sequence.pad_sequences(x_train, maxlen=max_length)
    x_val = sequence.pad_sequences(x_val, maxlen=max_length)
    return x_train, x_val


    

In [16]:
# preprocessed ready to train data

x_train,x_test = sequence_vectorize(df_train_reviews, df_test_reviews)


In [17]:
print(model_pretrained.evaluate(x_test, df_test_labels, batch_size = 64))
print(model_pretrained.evaluate(x_train, df_train_labels, batch_size = 128))


1000/1000 [==============================] - 5s 5ms/step
[0.9542548818588257, 0.454]
9000/9000 [==============================] - 21s 2ms/step
[0.9570432047843933, 0.45844444444444443]


In [24]:
model_pretrained.fit(x_train, df_train_labels, batch_size = 128,epochs = 100,shuffle=True)
model_pretrained.save('model_rnn_pretrained_trained2.h5')


Epoch 1/100
9000/9000 [==============================] - 60s 7ms/step - loss: 0.4244 - acc: 0.8148
Epoch 2/100
9000/9000 [==============================] - 60s 7ms/step - loss: 0.4229 - acc: 0.8164
Epoch 3/100
9000/9000 [==============================] - 60s 7ms/step - loss: 0.4132 - acc: 0.8189
Epoch 4/100
9000/9000 [==============================] - 59s 7ms/step - loss: 0.4129 - acc: 0.8213
Epoch 5/100
9000/9000 [==============================] - 59s 7ms/step - loss: 0.4069 - acc: 0.8249
Epoch 6/100
9000/9000 [==============================] - 59s 7ms/step - loss: 0.4043 - acc: 0.8224
Epoch 7/100
9000/9000 [==============================] - 59s 7ms/step - loss: 0.3987 - acc: 0.8261
Epoch 8/100
9000/9000 [==============================] - 58s 6ms/step - loss: 0.3952 - acc: 0.8282
Epoch 9/100
9000/9000 [==============================] - 59s 7ms/step - loss: 0.3944 - acc: 0.8268
Epoch 10/100
9000/9000 [==============================] - 59s 7ms/step - loss: 0.3820 - acc: 0.8334
Epoch 11/

9000/9000 [==============================] - 58s 6ms/step - loss: 0.1338 - acc: 0.9486
Epoch 84/100
9000/9000 [==============================] - 58s 6ms/step - loss: 0.1445 - acc: 0.9417
Epoch 85/100
9000/9000 [==============================] - 58s 6ms/step - loss: 0.1392 - acc: 0.9447
Epoch 86/100
9000/9000 [==============================] - 58s 6ms/step - loss: 0.1426 - acc: 0.9426
Epoch 87/100
9000/9000 [==============================] - 58s 6ms/step - loss: 0.1401 - acc: 0.9458
Epoch 88/100
9000/9000 [==============================] - 58s 6ms/step - loss: 0.1257 - acc: 0.9491
Epoch 89/100
9000/9000 [==============================] - 59s 7ms/step - loss: 0.1342 - acc: 0.9436
Epoch 90/100
9000/9000 [==============================] - 59s 7ms/step - loss: 0.1304 - acc: 0.9469
Epoch 91/100
9000/9000 [==============================] - 58s 6ms/step - loss: 0.1392 - acc: 0.9458
Epoch 92/100
9000/9000 [==============================] - 58s 6ms/step - loss: 0.1233 - acc: 0.9513
Epoch 93/100


In [20]:
model_pretrained.save('model_rnn_pretrained_trained.h5')

In [25]:
model_pretrained.evaluate(x_test, df_test_labels, batch_size = 64)

1000/1000 [==============================] - 4s 4ms/step


[1.2028902244567872, 0.7649999995231629]

In [23]:
model_pretrained.evaluate(x_train, df_train_labels, batch_size = 128)

9000/9000 [==============================] - 21s 2ms/step


[0.39871785895029704, 0.8224444443384806]